In [1]:
import sys
import os
import inspect
sys.path.append("..")


# get path of the current file
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# get the path of two directories above
parentdir = os.path.dirname(currentdir)
# add the parent directory to the sys path
sys.path.insert(0, parentdir)

import config
import pandas
from experiment import Experiment
from generation import Generation
from individual import Individual
from population import Population
from revolve2.experimentation.database import OpenMethod, open_database_sqlite
from sqlalchemy import select


dbengine = open_database_sqlite(
    "../"+config.DATABASE_FILE, open_method=OpenMethod.OPEN_IF_EXISTS
)

experiments = pandas.read_sql(
    select(Experiment.id.label("experiment_id")),
    dbengine,
)["experiment_id"].tolist()

table_names = pandas.read_sql("SELECT name FROM sqlite_master WHERE type='table'",
    dbengine,
)["name"].tolist()

/tmp/ipykernel_1912/2191279457.py:15: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
for name in table_names:
    df_exp = pandas.read_sql(f'PRAGMA table_info({name});', dbengine)["name"].tolist()
    print(f"{name}:\t", df_exp)

parameters:	 ['evolution_parameters', 'brain_multineat_parameters', 'body_multineat_parameters', 'id']
genotype:	 ['id', 'serialized_body', 'serialized_brain']
population:	 ['id']
experiment:	 ['rng_seed', 'steer', 'parameters_id', 'id']
individual:	 ['population_id', 'population_index', 'genotype_id', 'fitness', 'symmetry', 'xy_positions', 'id']
generation:	 ['experiment_id', 'generation_index', 'population_id', 'id']


In [3]:
params = pandas.read_sql(f'select * from parameters', dbengine).sort_values(by=["id"])

In [4]:
ts, steer = [], []
for i, row in params.iterrows():
    i_config = eval(row["evolution_parameters"].replace("false","False").replace("true", "True"))
    ts.append(i_config['tournament_size'])
    steer.append(i_config['steer'])

In [5]:
df_brain_params = []
for i, row in params.iterrows():
    df_row_brain = pandas.Series(eval(row["brain_multineat_parameters"].replace("false","False").replace("true", "True")))
    df_brain_params.append(df_row_brain)

df_brain_params = pandas.DataFrame(df_brain_params)
# df_brain_params

In [6]:
df_body_params = []
for i, row in params.iterrows():
    df_row_body = pandas.Series(eval(row["body_multineat_parameters"].replace("false","False").replace("true", "True")))
    df_body_params.append(df_row_body)

df_body_params = pandas.DataFrame(df_body_params)
# df_body_params

In [7]:
dif_df = pandas.DataFrame(columns=["exp_id","tournament_size","steer","body_mutation_rate","brain_mutation_rate"])
dif_df["exp_id"] = experiments
dif_df['tournament_size'] = ts
dif_df['steer'] = steer
dif_df["body_mutation_rate"] = df_body_params["OverallMutationRate"]
dif_df["brain_mutation_rate"] = df_brain_params["OverallMutationRate"]
dif_df

,exp_id,tournament_size,steer,body_mutation_rate,brain_mutation_rate
0,1,3,False,0.09,0.09
1,2,6,False,0.09,0.09
2,3,3,True,0.09,0.09
3,4,6,True,0.09,0.09
4,5,3,False,0.15,0.09
5,6,6,False,0.15,0.09
6,7,3,True,0.15,0.09
7,8,6,True,0.15,0.09
8,9,3,False,0.20,0.09
9,10,6,False,0.20,0.09
